In [2]:
import pandas as pd
import numpy as np
import os
import joblib
import tensorflow as tf

# V2 Import Style to fix Pylance warnings
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print(f"✅ TensorFlow Version: {tf.__version__}")
print("✅ All libraries imported successfully.")

C:\Users\fmala\AppData\Roaming\Python\Python313\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


✅ TensorFlow Version: 2.20.0
✅ All libraries imported successfully.


In [3]:
# Load the CSV files saved in data_preparation.ipynb
X_train = pd.read_csv('../data/X_train.csv')
y_train = pd.read_csv('../data/y_train.csv').values.ravel()

X_val = pd.read_csv('../data/X_val.csv')
y_val = pd.read_csv('../data/y_val.csv').values.ravel()

X_test = pd.read_csv('../data/X_test.csv')
y_test = pd.read_csv('../data/y_test.csv').values.ravel()

print(f"✅ Data Loaded. Training samples: {X_train.shape[0]}")

✅ Data Loaded. Training samples: 820


In [4]:
# 1. Initialize and Train
print("Training Baseline Model...")
baseline_model = LogisticRegression(max_iter=1000)
baseline_model.fit(X_train, y_train)

# 2. Evaluate
baseline_preds = baseline_model.predict(X_test)
baseline_acc = accuracy_score(y_test, baseline_preds)
print(f"📊 Baseline Accuracy: {baseline_acc:.2%}")

# 3. Save the Baseline
os.makedirs('../models', exist_ok=True)
joblib.dump(baseline_model, '../models/baseline_heart_model.pkl')
print("✅ Baseline model saved to models/baseline_heart_model.pkl")

Training Baseline Model...
📊 Baseline Accuracy: 78.64%
✅ Baseline model saved to models/baseline_heart_model.pkl


In [5]:
# 1. Define Architecture
model = models.Sequential([
    # Input layer matches the number of columns in X_train (13 features)
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Binary output (0 or 1)
])

# 2. Compile
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# 3. Train
print("\nTraining Neural Network...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=10,
    verbose=1
)

C:\Users\fmala\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training Neural Network...
Epoch 1/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8146 - loss: 0.5294 - val_accuracy: 0.8333 - val_loss: 0.4397
Epoch 2/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8512 - loss: 0.3792 - val_accuracy: 0.8039 - val_loss: 0.4078
Epoch 3/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8720 - loss: 0.3352 - val_accuracy: 0.8529 - val_loss: 0.3587
Epoch 4/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8695 - loss: 0.3179 - val_accuracy: 0.8235 - val_loss: 0.3599
Epoch 5/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8768 - loss: 0.2997 - val_accuracy: 0.8039 - val_loss: 0.3657
Epoch 6/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8805 - loss: 0.2849 - val_accuracy: 0.8627 - val_loss: 0.3325
Epoch 7/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8951 - loss: 0.2662 - val_accuracy: 0.8627 - val_loss: 0.3240
Epoch 8/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9122 - loss: 0.2519 - val_

In [6]:
# 1. Evaluate on Test Data
print("\nEvaluating Neural Network...")
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🚀 Final Neural Network Accuracy: {test_acc:.2%}")

# 2. Save the Model
model.save('../models/heart_disease_nn_model.keras')
print("✅ Neural Network saved to models/heart_disease_nn_model.keras")


Evaluating Neural Network...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9806 - loss: 0.1217
🚀 Final Neural Network Accuracy: 98.06%
✅ Neural Network saved to models/heart_disease_nn_model.keras


In [7]:
print("--- 🔍 DIAGNOSTIC REPORT ---")
print("1. The model expects columns in this EXACT order:")
print(list(X_train.columns))

print("\n2. Target Variable Check (1 usually means 'Disease'):")
print(pd.Series(y_train).value_counts())

--- 🔍 DIAGNOSTIC REPORT ---
1. The model expects columns in this EXACT order:
['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

2. Target Variable Check (1 usually means 'Disease'):
1    423
0    397
Name: count, dtype: int64


In [8]:
import pandas as pd
import numpy as np

print("--- 🧠 MODEL SANITY CHECK ---")

# 1. Get the first 10 people from the Test Set
sample_people = X_test.head(10)
sample_answers = y_test[:10]

# 2. Ask the model to predict their health
predictions = model.predict(sample_people)

# 3. Show the results side-by-side
print(f"\n{'True Label':<12} | {'Prediction':<12} | {'Result'}")
print("-" * 40)

for true, pred in zip(sample_answers, predictions):
    # Convert probability to 0 or 1
    p_val = pred[0]
    guess = 1 if p_val > 0.5 else 0
    
    # Check if the model is confident
    status = "✅ Correct" if true == guess else "❌ WRONG"
    print(f"{true:<12} | {p_val:.4f}       | {status}")

print("\n--- DIAGNOSIS ---")
avg_pred = np.mean(predictions)
print(f"Average Prediction Score: {avg_pred:.4f}")
if avg_pred > 0.95:
    print("🚨 FAILURE: The model is predicting '1' (High Risk) for EVERYONE.")
elif avg_pred < 0.05:
    print("🚨 FAILURE: The model is predicting '0' (Low Risk) for EVERYONE.")
else:
    print("✅ SUCCESS: The model is making varied predictions.")

--- 🧠 MODEL SANITY CHECK ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 932ms/step

True Label   | Prediction   | Result
----------------------------------------
0            | 0.0045       | ✅ Correct
0            | 0.0001       | ✅ Correct
1            | 1.0000       | ✅ Correct
1            | 1.0000       | ✅ Correct
0            | 0.0000       | ✅ Correct
1            | 1.0000       | ✅ Correct
0            | 0.0007       | ✅ Correct
0            | 0.0001       | ✅ Correct
1            | 1.0000       | ✅ Correct
1            | 0.9990       | ✅ Correct

--- DIAGNOSIS ---
Average Prediction Score: 0.5004
✅ SUCCESS: The model is making varied predictions.


In [9]:
# Find a patient the model is 100% sure is healthy
lowest_risk_idx = np.argmin(predictions)
healthy_patient = sample_people.iloc[lowest_risk_idx]

print("--- ✅ ENTER THESE VALUES IN YOUR APP ---")
print(healthy_patient)

--- ✅ ENTER THESE VALUES IN YOUR APP ---
age         0.062402
sex         1.000000
cp          0.000000
trestbps    0.479107
chol       -0.562371
fbs         0.000000
restecg     1.000000
thalach    -1.657533
exang       1.000000
oldpeak     3.855739
slope       0.000000
ca          0.000000
thal        3.000000
Name: 4, dtype: float64


In [10]:
import joblib

# 1. Load the Translator (Scaler)
scaler = joblib.load('../models/scaler.pkl')

# 2. Get the specific patient you found (Row 4)
# We need to reshape it to look like a single row of data
patient_data = healthy_patient.values.reshape(1, -1)
patient_df = pd.DataFrame(patient_data, columns=X_test.columns)

# 3. Un-Scale only the numerical columns
num_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
patient_df[num_cols] = scaler.inverse_transform(patient_df[num_cols])

# 4. Print the "Human Readable" Vitals
print("--- 🩺 ENTER THESE RAW NUMBERS INTO YOUR APP ---")
print(patient_df.iloc[0])

--- 🩺 ENTER THESE RAW NUMBERS INTO YOUR APP ---
age          55.0
sex           1.0
cp            0.0
trestbps    140.0
chol        217.0
fbs           0.0
restecg       1.0
thalach     111.0
exang         1.0
oldpeak       5.6
slope         0.0
ca            0.0
thal          3.0
Name: 0, dtype: float64
